In [1]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']


In [2]:

MODEL="sshleifer/distilbart-cnn-12-6"
HF_API_SUMMARY_BASE = f"{os.environ['HF_API_BASE']}/{MODEL}"
HF_API_SUMMARY_BASE


'https://api-inference.huggingface.co/models/sshleifer/distilbart-cnn-12-6'

In [3]:
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None,ENDPOINT_URL=HF_API_SUMMARY_BASE):
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

In [4]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris.
        Its base is square, measuring 125 metres (410 ft) on each side.
        During its construction, the Eiffel Tower surpassed the Washington
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure
        to reach a height of 300 metres. Due to the addition of a broadcasting
        aerial at the top of the tower in 1957, it is now taller than the
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the
        Eiffel Tower is the second tallest free-standing structure in France
        after the Millau Viaduct.''')

get_completion(text)

[{'generated_text': ' The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building in Paris . It is the second tallest free-standing structure in France after the Millau Viaduct . It was the first structure in the world to reach a height of 300 metres . The Eiffel Tower is now taller than the Chrysler Building in New York City by 5.2 metres (17 ft)'}]

## Gradio Interface

In [5]:

import gradio as gr
def summarize(input):
    output = get_completion(input)
    return output[0]['generated_text']

gr.close_all()
demo = gr.Interface(fn=summarize,
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result Summary", lines=3)],
                    title="Summarize Text",
                    description="Summarize text using GPT-3",)
demo.launch()
# demo.launch(share=True)
# demo.launch(share=True, server_port=int(os.environ['PORT1']))

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# Manual Model invocation

In [6]:
from transformers import pipeline

get_completion = pipeline("summarization", model=MODEL)

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

summarize(text)

2024-02-13 22:30:07.969554: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


' The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building in Paris . It is the second tallest free-standing structure in France after the Millau Viaduct . It was the first structure in the world to reach a height of 300 metres . The Eiffel Tower is now taller than the Chrysler Building in New York City by 5.2 metres (17 ft)'